In [1]:
import os
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
import json

In [ ]:
path = "../LLM/emg_datasets"
datasets = [
    os.path.join(path, x)
    for x in os.listdir(path)
    if os.path.isdir(os.path.join(path, x)) and "vs" in x
]
results = pd.DataFrame(
    columns=["Dataset", "Accuracy", "Precision", "Recall", "F1-Score", "ROC-AUC"]
)
for ds in datasets:
    train_df = pd.read_csv(os.path.join(ds, "train.csv"))
    test_df = pd.read_csv(os.path.join(ds, "test.csv"))

    num_classes = train_df["Class"].nunique()

    model = XGBClassifier()
    model.fit(train_df.drop(columns=["Patient", "Class"]), train_df["Class"])

    y_pred_test = model.predict(test_df.drop(columns=["Patient", "Class"]))
    y_pred_test_proba = model.predict_proba(test_df.drop(columns=["Patient", "Class"]))
    if num_classes == 2:
        results.loc[len(results)] = [
            ds.split("\\")[-1],
            accuracy_score(test_df["Class"], y_pred_test),
            precision_score(test_df["Class"], y_pred_test),
            recall_score(test_df["Class"], y_pred_test),
            f1_score(test_df["Class"], y_pred_test),
            roc_auc_score(test_df["Class"], y_pred_test),
        ]

    else:

        results.loc[len(results)] = [
            ds.split("\\")[-1],
            accuracy_score(test_df["Class"], y_pred_test),
            precision_score(test_df["Class"], y_pred_test, average="macro"),
            recall_score(test_df["Class"], y_pred_test, average="macro"),
            f1_score(test_df["Class"], y_pred_test, average="macro"),
            roc_auc_score(test_df["Class"], y_pred_test_proba, multi_class="ovo"),
        ]


results

,Dataset,Accuracy,Precision,Recall,F1-Score,ROC-AUC
0,als_vs_control_and_myopathy_dataset,0.966292,0.881944,0.888112,0.885017,0.933888
1,als_vs_myopathy_dataset,0.973354,0.949833,0.993007,0.970940,0.975197
2,control_vs_als_and_myopathy_dataset,0.894280,0.795205,0.909875,0.848684,0.898309
3,control_vs_als_dataset,0.962329,0.893543,0.895105,0.894323,0.935999
4,control_vs_als_vs_myopathy_dataset,0.893258,0.854946,0.893592,0.871347,0.974569
5,control_vs_myopathy_dataset,0.909689,0.729977,0.906250,0.808619,0.908428
6,myopathy_vs_control_and_als_dataset,0.914198,0.718527,0.859375,0.782665,0.892795
